In [1]:
import h5py
import pyccl
import scipy
import numpy
import json

In [2]:
# Load in Data

PATH = '/pscratch/sd/y/yhzhang/TensorCloud/'
DATA_PATH = PATH + 'DATA/'
CODE_PATH = PATH + 'CODE/'
PLOT_PATH = PATH + 'PLOT/'

BIN_SIZE = 4
GRID_SIZE = 72
LATENT_SIZE = 32
DATA_SIZE = 100000
COLOR = ['blue', 'green', 'orange', 'red']

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as FILE:
    
    Z_GRID = numpy.array(FILE['realizations/z'][:GRID_SIZE + 1], dtype = 'double')
    GRID = numpy.array(FILE['realizations/pdfs'][:, :BIN_SIZE, :GRID_SIZE + 1], dtype = 'double')

In [3]:
# COSMOLOGY

with open(DATA_PATH + 'COSMO.json', 'r') as FILE:
    COSMO = json.load(FILE)
    
COSMO_CCL = pyccl.cosmology.Cosmology(
    h=COSMO['H'],
    w0=COSMO['W0'],
    wa=COSMO['WA'],
    n_s=COSMO['NS'],
    m_nu=COSMO['MNU'],
    Neff=COSMO['NEFF'],
    T_CMB=COSMO['TCMB'],
    mass_split='single',
    sigma8=COSMO['SIGMA8'],
    Omega_k=COSMO['OMEGAK'],
    Omega_c=COSMO['OMEGAC'],
    Omega_b=COSMO['OMEGAB'],
    mg_parametrization=None,
    matter_power_spectrum='halofit',
    transfer_function='boltzmann_camb',
    extra_parameters={
        'camb': {'kmax': 10000, 'lmax': 10000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [4]:
# PMM, matter-matter-power-spectra

PSI_GRID = numpy.mean(GRID, axis = 0)
SIGMA_GRID = numpy.std(GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + Z_GRID))
CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + Z_GRID)) * COSMO['H'] * 100000 / scipy.constants.c

ELL_SIZE = 100
ELL_DATA = numpy.logspace(1, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where = CHI_MESH > 0))

PMM_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):

    PMM_GRID[:,GRID_INDEX] = COSMO_CCL.linear_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

In [5]:
#numerical integral

def integralI7(chi1, chi2, chi3, chi4, chi5, power1, power2, redshift1, redshift2):

    def formulan(chi):
        
        result = (chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1)

        result = result * (- chi + (chi5 - chi4) / 2 + chi * chi4 * numpy.log(chi5 / chi4) / (chi5 - chi4))

        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)**2

        return result

    def formula0(chi):
        
        result = (chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1)

        result = result * (- chi + (chi5 - chi4) / 2 + chi * chi4 * numpy.log(chi5 / chi4) / (chi5 - chi4))

        result = result * (((chi/chi2)**3)*power2)  
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)**2

        return result
    
    if chi1 == 0:
        
        integral, error = scipy.integrate.quad_vec(f = formula0, a = chi1, b = chi2)
        
    else:
        
        integral, error = scipy.integrate.quad_vec(f = formulan, a = chi1, b = chi2)

    return integral

In [6]:
#coefficient equation

def coefficientJ7(chi1, chi2, chi3, chi4, chi5, power1, power2, redshift1, redshift2):

    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    c = (chi5-chi4)/(2*chi2)
    d = chi4*numpy.log(chi5/chi4)/(chi5-chi4)-1
    y = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)

    if a == 1:

        formula = (1 / (5040 * b)) * (b * (2 * d * (432 + z * ( - 129 + 17 * z) + 12 * b * (21 + ( - 7 + z) * z)) + 3 * c * (350 + z * ( - 124 + 19 * z) + 14 * b * (15 + ( - 6 + z) * z))) - 
6 * (1 + b) * (5 * d * (28 + ( - 8 + z) * z) + 8 * c * (21 + ( - 7 + z) * z)) * numpy.log(1 + b))

    else: 

        formula = (1 / (25200 * a**4 * b)) * (a * b * ( - 420 * (3 * c + d) * y * z**2 + 70 * a**2 * ( - 30 * (2 * c + d) * y - 6 * (5 * c * (4 + y) + 2 * d * (5 + y)) * z + (d * ( - 6 + y) + 3 * c * ( - 5 + y)) * z**2) + 
210 * a * z * (8 * d * y + 10 * c * z + d * (4 + y) * z + c * y * (20 + 3 * z)) + a**6 * d * ( - 5950 + 4095 * y + 42 * (195 - 74 * z) * z + 4 * y * z * ( - 1554 + 625 * z)) + 
35 * a**3 * (c * (60 * (6 + y) - 20 * ( - 6 + y) * z + ( - 10 + 3 * y) * z**2) + d * (240 - 4 * ( - 15 + z) * z + y * (30 + ( - 8 + z) * z))) + 
7 * a**5 * (c * (1500 - 850 * y + 6 * y * (195 - 74 * z) * z + 5 * z * ( - 340 + 117 * z)) + d * ( - 25 * ( - 64 + 35 * y + 70 * z) + 2 * z * (y * (594 - 224 * z) + 297 * z) + 
30 * b * ( - 30 + 20 * y + 5 * (8 - 3 * z) * z + 6 * y * z * ( - 5 + 2 * z)))) + 
7 * a**4 * (d * (50 * ( - 12 + y) - 20 * ( - 5 + y) * z + ( - 10 + 3 * y) * z**2 - 150 * b * ( - 4 * (3 + ( - 3 + z) * z) + y * (6 + z * ( - 8 + 3 * z)))) + 
c * (100 * ( - 9 + y + 2 * z) + z * ( - 25 * z + y * ( - 50 + 9 * z)) - 150 * b * ( - 4 * (3 + ( - 3 + z) * z) + y * (6 + z * ( - 8 + 3 * z)))))) - 
420 * (( - 1 + a)**3 * b * (5 * a**2 * (6 * a * c - 4 * ( - 1 + a) * a * d - ((2 + 4 * a) * c + d + (2 - 3 * a) * a * d) * y) + 
2 * a * ( - 10 * a * (1 + 2 * a) * c + 5 * ( - 1 + a) * a * (1 + 3 * a) * d + 5 * (1 + a * (2 + 3 * a)) * c * y - 2 * ( - 1 + a) * (1 + 3 * a + 6 * a**2) * d * y) * z + 
(5 * a * (1 + a * (2 + 3 * a)) * c - 2 * ( - 1 + a) * a * (1 + 3 * a + 6 * a**2) * d - (3 * (1 + a * (2 + a * (3 + 4 * a))) * c + d + a * (2 + a * (3 + 2 * (2 - 5 * a) * a)) * d) * y) * z**2) * numpy.log(1 - a) + a**5 * (1 + b) * (d * (20 * (3 + ( - 3 + a) * a) - 5 * (6 + a * ( - 8 + 3 * a)) * y + 2 * ( - 30 + 5 * (8 - 3 * a) * a + 20 * y + 6 * a * ( - 5 + 2 * a) * y) * z + 
(20 - 15 * y + 2 * a * ( - 15 + 6 * a + 12 * y - 5 * a * y)) * z**2) + c * (20 * (3 + ( - 3 + z) * z) + a * ( - 30 + 20 * y + 5 * (8 - 3 * z) * z + 6 * y * z * ( - 5 + 2 * z)) - 
5 * y * (6 + z * ( - 8 + 3 * z)))) * numpy.log(1 + b)))

    coefficient = chi2**3 * power2 * (1 + redshift2)**2 * formula

    return coefficient

In [7]:
#Case1 test: For chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]
CHI4 = CHI_GRID[GRID_SIZE - 1]
CHI5 = CHI_GRID[GRID_SIZE]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

1461.30684999374 1497.8183354014277 1534.1133113296617 2616.3612383175455 2645.867010982371 49919.45246058225 48958.62248873209 0.36 0.37


In [8]:
#Case1 test: For chi1 > 0

INTEGRAL = integralI7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(10412295.123159349, 10412295.123117058, -4.061639913288673e-12)

In [9]:
#Case2 test: For chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]
CHI4 = CHI_GRID[GRID_SIZE - 1]
CHI5 = CHI_GRID[GRID_SIZE]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.42674493673601 88.64121385461215 2616.3612383175455 2645.867010982371 0.0 2114.2567965927065 0.0 0.01


In [10]:
#Case2 test: For chi1 = 0

INTEGRAL = integralI7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(3440764.2937925486, 3440764.2937925505, 4.440892098500626e-16)